# enum - Python 标准库之 枚举

## 什么是 Enum, 什么时候用 Enum?

1. 定义一堆可以被归为一类的常量的时候.

In [99]:
import typing as T
from rich import print
import enum

## Enum 的基本概念

In [101]:
# 这个 red 和 blue 是 name
class ColorEnum(enum.Enum):
    red = 1
    blue = 2

In [103]:
# 这个 ColorEnum.red 是 member
ColorEnum.red

<ColorEnum.red: 1>

In [104]:
# 你也可以用 [] 语法来访问 member
ColorEnum["red"]

<ColorEnum.red: 1>

In [105]:
# 一个 member 的具体的值是 value, 可以用 ColorEnum.red.value 这样的语法来访问
ColorEnum.red.value

1

## Restricted Enum subclassing - 继承是有限制的

Reference:

- [restricted-enum-subclassing](https://docs.python.org/3/library/enum.html#restricted-enum-subclassing)

一个 ``Enum`` 类可以包含:

1. 任意多个 Mixin 类, 可以没有, 且通常没有, 除非你要自行扩展功能
2. 至多一个具体的数据类型, 可以没有, 且通常没有
3. 有且一个 Enum 基类, 可以是 ``enum.Enum``, ``enum.IntEnum``, ``enum.StrEnum`` 之一

继承的顺序必须是按照上面来的.

**并且如果你要继承其他的 Enum 类的话, 父类不能够定义任何成员**

例如下面的例子是行不通的.

In [5]:
class SomeColor(enum.Enum):
    red = 1


class MoreColor(SomeColor):
    blue = 2

TypeError: MoreColor: cannot extend enumeration 'SomeColor'

In [7]:
# 但是这样可以
class Foo(enum.Enum):
    def some_behavior(self):
        pass


class Bar(Foo):
    HAPPY = 1
    SAD = 2


Bar.HAPPY

<Bar.HAPPY: 1>

## Data Type in subclassing

这里详细说一下继承时有 data type 的情况. 这里指的是 ``class MyEnum(a_class, enum.Enum):`` 的情况. 官方文档是这么说的:

    When subclassing other data types, such as int or str, with an Enum, all values after the = are passed to that data type’s constructor.

这个意思就是所有在等号后面的数据, 都会被传递给 ``a_class`` 这个 constructor 函数中去. 注意, 这里的 constructor 指的是 ``__new__`` 方法, 而不是 initializer ``__init__`` 方法. 我们来看下面的官方例子.

In [52]:
# class MyEnum(int, enum.Enum) 和 class MyEnum(enum.IntEnum) 等效
class MyEnum(int, enum.Enum):
    example = "11", 16


print(int("11", 16))  # 实际上是 int("11", base=16)
print(type(MyEnum.example.value))
print(MyEnum.example.value)  # 可以看出 example = 的内容实际上是 int("11", 16)

17

<class 'int'>

17

下面这个例子中我们只定义了 ``__init__``, 没有定义 ``__new__``, 所以最后的 value 还是原来的字符串. 这是一个 **错误示范**.

In [76]:
class Apple:
    def __init__(self, apple_name: str):
        self.apple_name = apple_name

    def apple_special_method(self):
        return "this is apple special method"


class AppleEnum(Apple, enum.Enum):
    red = "red_apple"
    green = "green_apple"

In [77]:
print(type(AppleEnum.red.value))

<class 'str'>

In [78]:
print(AppleEnum.red.value)  # type hint 不能发现 .value 其实是 Apple, 因为它真的不是

red_apple

In [79]:
AppleEnum.red.value.apple_special_method()

AttributeError: 'str' object has no attribute 'apple_special_method'

下面这个是 **正确示范**

In [80]:
class Orange:
    def __init__(self, orange_name: str):
        self.orange_name = orange_name

    def __new__(cls, orange_name: str, **kwargs):
        orange = object.__new__(cls)
        orange.__init__(orange_name)
        return orange

    def __repr__(self):
        return f"Orange(orange_name={self.orange_name!r})"

    def orange_special_method(self):
        return "this is orange special method"


class OrangeEnum(Orange, enum.Enum):
    big = "big_orange"

In [81]:
print(type(OrangeEnum.big))

<enum 'OrangeEnum'>

In [82]:
print(OrangeEnum.big)  # **type hint 能发现 .big 其实是 Orange**

OrangeEnum.big

In [83]:
print(OrangeEnum.big.orange_special_method())  # 虽然上面打印出来是一个枚举, 但实际上可以用作 Orange 对象

this is orange special method

In [84]:
print(type(OrangeEnum.big.value))  # .value 还是 Orange 对象

<class '__main__.Orange'>

In [85]:
print(OrangeEnum.big.value)  # type hint 不能发现 .value 其实是 Orange

Orange(orange_name='big_orange')

In [86]:
isinstance(OrangeEnum.big, OrangeEnum)

True

In [89]:
# 你既可以让迭代器返回: 成员, 也可以返回 name, 也可以返回 value
for orange in OrangeEnum:
    print(orange.name, orange.value)

big Orange(orange_name='big_orange')

In [91]:
OrangeEnum["big"].orange_special_method()

'this is orange special method'

In [95]:
# 测试一个 name 在不在 Enum 里面
OrangeEnum["invalid"]

KeyError: 'invalid'

In [98]:
# 测试一个 member 在不在 Enum 里面
OrangeEnum.big in OrangeEnum

True

**这个模式的功能**

1. ensure that all members are of that type. 确保成员对象都是这个类. 如果构造器能构造那么就当做是这个类, 反之不行.
2. make the members directly usable as that type. 成员 (也就是 ``Orange.big`` 语法) 可以直接当做对象类来用, 免去了用 ``.value`` 的麻烦. 这在 成员不重要, 而值重要的情况下非常有用!

In [75]:
# ensure that all members are of that type
class OrangeEnum(Orange, enum.Enum):
    big = "big_orange"
    small = 1, 2, 3

TypeError: __new__() takes 2 positional arguments but 4 were given

## Cookbook - 技巧大全

### Typed Enum

In [201]:
import attr


@attr.s
class WholeSale:
    business_name: str = attr.ib()

    def wholesale_special_method(self):
        return "this is wholesale special method"


class TypedEnumMixin:
    @classmethod
    def has_name(cls, name: str) -> bool:
        try:
            _ = cls[name]
            return True
        except KeyError:
            return False

    @classmethod
    def has_member(cls, member) -> bool:
        return member in cls

    _all_values = None

    @classmethod
    def has_value(cls, value) -> bool:
        if cls._all_values is None:
            cls._all_values = [member._value_ for member in cls]
        print(cls._all_values, cls._all_values[0], type(cls._all_values[0]))
        return value in cls._all_values


class WholeSaleEnum(TypedEnumMixin, WholeSale, enum.Enum):
    walmart = "walmart"


class OtherWholeSaleEnum(enum.Enum):
    costco = "costco"

# print(WholeSale(business_name="walmart") in list(WholeSaleEnum._value2member_map_))
print(type(WholeSaleEnum.walmart))
print(WholeSaleEnum)
# WholeSaleEnum.has_value(WholeSale(business_name="walmart"))
# for wholesale in WholeSaleEnum:
#     print(attr.asdict(wholesale))

<enum 'WholeSaleEnum'>

<enum 'WholeSaleEnum'>

['walmart']
walmart <class 'str'>

False

In [169]:
WholeSaleEnum.walmart.wholesale_special_method()

'this is wholesale special method'

In [170]:
WholeSaleEnum.walmart.business_name

'walmart'

In [171]:
WholeSaleEnum.has_name("walmart")

True

In [172]:
WholeSaleEnum.has_name("costco")

False

In [173]:
WholeSaleEnum.has_member(WholeSaleEnum.walmart)

True

In [174]:
WholeSaleEnum.has_member(OtherWholeSaleEnum.costco)

False

In [175]:
WholeSaleEnum.has_value(WholeSale(business_name="walmart"))

['walmart']
walmart <class 'str'>

False